In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from zipfile import ZipFile
file_name = "drive/My Drive/ashrae-energy-prediction.zip"
with ZipFile(file_name, 'r') as zip:  
    zip.printdir() 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
building_metadata.csv                          2019-10-10 17:20:04        45527
sample_submission.csv                          2019-10-10 17:20:12    447562511
test.csv                                       2019-10-10 17:20:46   1462461085
train.csv                                      2019-10-10 17:22:42    678616640
weather_test.csv                               2019-10-10 17:23:40     14787908
weather_train.csv                              2019-10-10 17:23:42      7450075
Extracting all the files now...
Done!


In [4]:
!pip install category_encoders

     |████████████████████████████████| 102kB 2.4MB/s 


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from IPython import display
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import category_encoders as ce

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
      print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
weather_train = pd.read_csv("weather_train.csv")
weather_test = pd.read_csv("weather_test.csv")
building = pd.read_csv("building_metadata.csv")
sub = pd.read_csv("sample_submission.csv")
gc.collect()

0

In [0]:
le = LabelEncoder()
building["primary_use"] = le.fit_transform(building["primary_use"])

In [0]:
# Interpolation for missing values
weather_train = weather_train.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))
gc.collect()
weather_test = weather_test.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))

In [0]:
train = train.merge(building, on = "building_id", how = "left")
test = test.merge(building, on = "building_id", how = "left")

In [0]:
train = train.merge(weather_train, on = ["site_id", "timestamp"], how='left')
test = test.merge(weather_test, on = ["site_id", "timestamp"], how='left')

In [12]:
train["wind_direction"].nunique()/len(train) * 100

0.0026859780076275842

In [0]:
# Handling missing values
# Training set
train["year_built"].fillna(-1,inplace=True)
train["floor_count"].fillna(-1,inplace=True)
train["air_temperature"].fillna(train["air_temperature"].mean(),inplace=True)
train["dew_temperature"].fillna(train["dew_temperature"].mean(),inplace=True)
train["wind_speed"].fillna(train["wind_speed"].mean(),inplace=True)
train["wind_direction"].fillna(train["wind_direction"].mode(),inplace=True)
train.fillna(-999,inplace=True)
# Test set
test["year_built"].fillna(-1,inplace=True)
test["floor_count"].fillna(-1,inplace=True)
test["air_temperature"].fillna(test["air_temperature"].mean(),inplace=True)
test["dew_temperature"].fillna(test["dew_temperature"].mean(),inplace=True)
test["wind_direction"].fillna(test["wind_direction"].mean(),inplace=True)
test["wind_direction"].fillna(test["wind_direction"].mode(),inplace=True)
test.fillna(-999,inplace=True)

In [14]:
# Feature engineering
train["day"] = pd.to_datetime(train["timestamp"]).dt.day
train["month"] = pd.to_datetime(train["timestamp"]).dt.month
train["dayofweek"] = pd.to_datetime(train["timestamp"]).dt.dayofweek
train["hour"] = pd.to_datetime(train["timestamp"]).dt.hour
test["day"] = pd.to_datetime(test["timestamp"]).dt.day
test["month"] = pd.to_datetime(test["timestamp"]).dt.month
test["dayofweek"] = pd.to_datetime(test["timestamp"]).dt.dayofweek
test["hour"] = pd.to_datetime(test["timestamp"]).dt.hour

a_train = train["site_id"].astype(str) + "_" + train["building_id"].astype(str)
a_test = test["site_id"].astype(str) + "_" + test["building_id"].astype(str)
pbar = tqdm(total=8)
woe1 = ce.woe.WOEEncoder()
train["site_id-building_id_yearbuilt_woe"] = woe1.fit_transform(a_train,train["year_built"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
test["site_id-building_id_yearbuilt_woe"] = woe1.transform(a_test,test["year_built"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
woe2 = ce.woe.WOEEncoder()
train["site_id-building_id_floorcount_woe"] = woe2.fit_transform(a_train,train["floor_count"].apply(lambda x: 0 if x==-1 else 1)) 
pbar.update(1)
test["site_id-building_id_floorcount_woe"] = woe2.transform(a_test,test["floor_count"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
woe3 = ce.woe.WOEEncoder()
train["primaryuse_yearbuilt_woe"] = woe3.fit_transform(train["primary_use"].astype(str),train["year_built"].apply(lambda x: 0 if x==-1 else 1)) 
pbar.update(1)
test["primaryuse_yearbuilt_woe"] = woe3.transform(test["primary_use"].astype(str),test["year_built"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
woe4 = ce.woe.WOEEncoder()
train["primaryuse_floorcount_woe"] = woe4.fit_transform(train["primary_use"].astype(str),train["floor_count"].apply(lambda x: 0 if x==-1 else 1)) 
pbar.update(1)
test["primaryuse_floorcount_woe"] = woe4.transform(test["primary_use"].astype(str),test["floor_count"].apply(lambda x: 0 if x==-1 else 1))
pbar.update(1)
pbar.close()

100%|██████████| 8/8 [03:15<00:00, 29.20s/it]


In [0]:
del weather_train
del weather_test
del building

In [16]:
gc.collect()

0

In [17]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
sub = reduce_mem_usage(sub)

Mem. usage decreased to 1777.49 Mb (60.5% reduction)
Mem. usage decreased to 2346.19 Mb (70.5% reduction)
Mem. usage decreased to 198.83 Mb (68.7% reduction)


In [18]:
gc.collect()

0

In [0]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
train['is_holiday'] = (pd.to_datetime(train['timestamp']).dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
test['is_holiday'] = (pd.to_datetime(test['timestamp']).dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

train.loc[(train['dayofweek'] == 5) | (train['dayofweek'] == 6) , 'is_holiday'] = 1
test.loc[(test['dayofweek']) == 5 | (test['dayofweek'] == 6) , 'is_holiday'] = 1

In [0]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [0]:
def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=[i for i in range(0,16)]
    return arr[(val % 16)]

In [0]:
train['wind_direction'] = train['wind_direction'].apply(degToCompass)
test['wind_direction'] = test['wind_direction'].apply(degToCompass)

In [0]:
target = np.log1p(train["meter_reading"])
train.drop(["timestamp", "meter_reading"], axis = 1, inplace=True)
test.drop(["timestamp", "row_id"], axis = 1, inplace=True)

In [24]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
sub = reduce_mem_usage(sub)

Mem. usage decreased to 890.63 Mb (13.0% reduction)
Mem. usage decreased to 1869.00 Mb (13.0% reduction)
Mem. usage decreased to 198.83 Mb (0.0% reduction)


In [25]:
train.shape, target.shape, test.shape

((19869988, 23), (19869988,), (41697600, 23))

In [26]:
!pip install catboost
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split, KFold
import random
from tqdm import tqdm

     |████████████████████████████████| 62.9MB 155kB/s 


In [0]:
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2_root', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.5,
    'subsample':0.25,
    'subample_freq': 1,
    'lambda_l1': 1,  
    'lambda_l2': 1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

In [0]:
folds=KFold(n_splits=10,random_state=1)

In [29]:
test_pred1=[]
train_pred1=np.empty((0,1),float)
for train_indices,val_indices in folds.split(train,target):
    x_train,x_valid=train.iloc[train_indices],train.iloc[val_indices]
    y_train,y_valid=target.iloc[train_indices],target.iloc[val_indices]
    dtrain = lgb.Dataset(x_train, label=y_train)
    dvalid = lgb.Dataset(x_valid, label=y_valid)
    model = lgb.train(lgb_params,
                dtrain,
                num_boost_round=500,
                valid_sets=dvalid,
                early_stopping_rounds=5,
                verbose_eval=100)
    i=0
    res=[]
    step_size = 500000
    for j in tqdm(range(int(np.ceil(x_valid.shape[0]/500000)))):
        res.append(np.expm1(model.predict(x_valid.iloc[i:i+step_size],num_iteration=model.best_iteration)))
        i+=step_size
    train_pred1=np.append(train_pred1,np.concatenate(res))
    i=0
    res=[]
    step_size = 500000
    for j in tqdm(range(int(np.ceil(test.shape[0]/500000)))):
        res.append(np.expm1(model.predict(test.iloc[i:i+step_size],num_iteration=model.best_iteration)))
        i+=step_size
    test_pred1.append(np.concatenate(res))
test_pred1=np.array(test_pred1).T.mean(axis=1)
train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)
gc.collect()

Training until validation scores don't improve for 5 rounds.
[100]	valid_0's l1: 0.710982	valid_0's rmse: 1.14342
[200]	valid_0's l1: 0.598661	valid_0's rmse: 1.03462


  0%|          | 0/4 [00:00<?, ?it/s]

Early stopping, best iteration is:
[231]	valid_0's l1: 0.588409	valid_0's rmse: 1.02609


100%|██████████| 84/84 [01:44<00:00,  1.06s/it]


Training until validation scores don't improve for 5 rounds.
[100]	valid_0's l1: 0.667599	valid_0's rmse: 1.03987
[200]	valid_0's l1: 0.579695	valid_0's rmse: 0.951822
[300]	valid_0's l1: 0.537734	valid_0's rmse: 0.908843
[400]	valid_0's l1: 0.50804	valid_0's rmse: 0.878642


...

Training until validation scores don't improve for 5 rounds.
[100]	valid_0's l1: 0.668577	valid_0's rmse: 1.03782
[200]	valid_0's l1: 0.605758	valid_0's rmse: 0.977433


  0%|          | 0/4 [00:00<?, ?it/s]

Early stopping, best iteration is:
[219]	valid_0's l1: 0.59556	valid_0's rmse: 0.96809


100%|██████████| 84/84 [01:37<00:00,  1.00s/it]


60

In [30]:
test_pred2=[]
train_pred2=np.empty((0,1),float)
for train_indices,val_indices in folds.split(train,target):
    x_train,x_valid=train.iloc[train_indices],train.iloc[val_indices]
    y_train,y_valid=target.iloc[train_indices],target.iloc[val_indices]
    dtrain = Pool(x_train, label=y_train)
    dvalid = Pool(x_valid, label=y_valid)
    model = CatBoostRegressor(loss_function="MAE",
                            eval_metric="RMSE",
                            task_type="GPU",
                            learning_rate=0.5,
                            iterations=1500,
                            random_seed=1,
                            od_type="Iter",
                            depth=12,
                            early_stopping_rounds=20,
                            border_count=64)
    fit_model = model.fit(dtrain,
                            eval_set=dvalid,
                            use_best_model=True,
                            verbose=100)
    i=0
    res=[]
    step_size = 500000
    for j in tqdm(range(int(np.ceil(x_valid.shape[0]/500000)))):
        res.append(np.expm1(fit_model.predict(x_valid.iloc[i:i+step_size])))
        i+=step_size
    train_pred2=np.append(train_pred2,np.concatenate(res))
    i=0
    res=[]
    step_size = 500000
    for j in tqdm(range(int(np.ceil(test.shape[0]/500000)))):
        res.append(np.expm1(fit_model.predict(test.iloc[i:i+step_size])))
        i+=step_size
    test_pred2.append(np.concatenate(res))
    del res
    del fit_model
    del model
    del x_train
    del x_valid
    del y_train
    del y_valid
    del dtrain
    del dvalid
    gc.collect()
test_pred2=np.array(test_pred2).T.mean(axis=1)
train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)
gc.collect()

0:	learn: 4.5163208	test: 4.4740048	best: 4.4740048 (0)	total: 154ms	remaining: 3m 50s
100:	learn: 1.1620846	test: 1.2265915	best: 1.2265915 (100)	total: 14.2s	remaining: 3m 16s
200:	learn: 1.0363946	test: 1.1250288	best: 1.1250288 (200)	total: 28.1s	remaining: 3m 1s
300:	learn: 0.9458333	test: 1.0582270	best: 1.0582270 (300)	total: 41.7s	remaining: 2m 46s
400:	learn: 0.9043805	test: 1.0351135	best: 1.0351135 (400)	total: 55.5s	remaining: 2m 31s
500:	learn: 0.8783782	test: 1.0191112	best: 1.0190724 (499)	total: 1m 9s	remaining: 2m 18s
600:	learn: 0.8592588	test: 1.0106477	best: 1.0106477 (600)	total: 1m 23s	remaining: 2m 4s
700:	learn: 0.8375836	test: 1.0010365	best: 1.0010365 (700)	total: 1m 36s	remaining: 1m 50s
bestTest = 1.001036517
bestIteration = 700
Shrink model to first 701 iterations.


100%|██████████| 84/84 [01:48<00:00,  1.08s/it]


0:	learn: 4.5145079	test: 4.4887313	best: 4.4887313 (0)	total: 150ms	remaining: 3m 45s
100:	learn: 1.1701736	test: 1.1828669	best: 1.1828669 (100)	total: 14.2s	remaining: 3m 16s
200:	learn: 1.0431455	test: 1.0521166	best: 1.0521166 (200)	total: 27.9s	remaining: 3m
300:	learn: 0.9568548	test: 0.9744947	best: 0.9744947 (300)	total: 41.5s	remaining: 2m 45s
400:	learn: 0.9102839	test: 0.9358399	best: 0.9358399 (400)	total: 55.2s	remaining: 2m 31s
500:	learn: 0.8763715	test: 0.9162979	best: 0.9162979 (500)	total: 1m 8s	remaining: 2m 17s
bestTest = 0.9160186643
bestIteration = 522
Shrink model to first 523 iterations.


100%|██████████| 84/84 [01:32<00:00,  1.10it/s]


0:	learn: 4.5171616	test: 4.4577622	best: 4.4577622 (0)	total: 149ms	remaining: 3m 44s
100:	learn: 1.1660100	test: 1.1914459	best: 1.1914459 (100)	total: 14.1s	remaining: 3m 15s
200:	learn: 1.0341833	test: 1.0734189	best: 1.0734189 (200)	total: 27.9s	remaining: 3m
300:	learn: 0.9581567	test: 1.0055739	best: 1.0055739 (300)	total: 41.5s	remaining: 2m 45s
400:	learn: 0.9107726	test: 0.9709062	best: 0.9709062 (400)	total: 55.2s	remaining: 2m 31s
500:	learn: 0.8791061	test: 0.9500003	best: 0.9500003 (500)	total: 1m 8s	remaining: 2m 17s
600:	learn: 0.8555453	test: 0.9373711	best: 0.9373711 (600)	total: 1m 22s	remaining: 2m 3s
bestTest = 0.9366106759
bestIteration = 636
Shrink model to first 637 iterations.


...

bestTest = 0.9247981487
bestIteration = 386
Shrink model to first 387 iterations.


100%|██████████| 84/84 [01:22<00:00,  1.23it/s]


0:	learn: 4.5131202	test: 4.5044951	best: 4.5044951 (0)	total: 147ms	remaining: 3m 39s
100:	learn: 1.1619045	test: 1.0997817	best: 1.0997817 (100)	total: 14.2s	remaining: 3m 16s
200:	learn: 1.0373345	test: 0.9969240	best: 0.9969240 (200)	total: 28s	remaining: 3m 1s
300:	learn: 0.9486518	test: 0.9512241	best: 0.9512241 (300)	total: 41.6s	remaining: 2m 45s
400:	learn: 0.9088961	test: 0.9272285	best: 0.9265044 (395)	total: 55.4s	remaining: 2m 31s
500:	learn: 0.8809900	test: 0.9175616	best: 0.9169779 (497)	total: 1m 8s	remaining: 2m 17s
600:	learn: 0.8555445	test: 0.9087439	best: 0.9085361 (595)	total: 1m 22s	remaining: 2m 3s
bestTest = 0.9057943101
bestIteration = 650
Shrink model to first 651 iterations.


100%|██████████| 84/84 [01:43<00:00,  1.02s/it]


0

In [31]:
gc.collect()

0

In [0]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

In [0]:
from sklearn.linear_model import HuberRegressor as r

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras
from keras.wrappers.scikit_learn import KerasRegressor

def baseline_model():
  # create model
  model = Sequential()
  model.add(Dense(2000, input_dim=2, kernel_initializer='normal', activation='relu'))
  model.add(Dense(500, kernel_initializer='normal',activation='relu'))
  model.add(Dense(100, kernel_initializer='normal',activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam())
  return model

In [58]:
#model = r(epsilon=5,fit_intercept=False)
model = KerasRegressor(build_fn=baseline_model, epochs=20, batch_size=10000, verbose=1)
model.fit(df.values,target.values.reshape(target.shape[0],1))
#model.coef_
i=0
res=[]
step_size = 500000
for j in tqdm(range(int(np.ceil(df_test.shape[0]/500000)))):
    res.append(np.expm1(model.predict(df_test.iloc[i:i+step_size], verbose=0)))
    i+=step_size
results = np.concatenate(res)

Epoch 1/20
19869988/19869988 [==============================] - 34s 2us/step - loss: 1890198.2610
Epoch 2/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 46.7614
Epoch 3/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 1499.9277
Epoch 4/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 11.7084
Epoch 5/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 9816.9462
Epoch 6/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 3.3126
Epoch 7/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 1.8209
Epoch 8/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 1.3393
Epoch 9/20
19869988/19869988 [==============================] - 33s 2us/step - loss: 1.0600
Epoch 10/20
19869988/19869988 [==============================] - 34s 2us/step - loss: 0.9837
Epoch 11/20
19869988/19869988 [==============================] - 

100%|██████████| 84/84 [00:22<00:00,  4.53it/s]


In [59]:
#sub["meter_reading"] = ((test_pred1 * model.coef_[0]) + (test_pred2 * model.coef_[1]))*1000
sub["meter_reading"] = results
sub["meter_reading"][sub["meter_reading"] < 0] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
sub["meter_reading"][sub["meter_reading"] == np.inf] = sub["meter_reading"].quantile(0.999)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
sub.head()

,row_id,meter_reading
0,0,148.382812
1,1,70.850471
2,2,9.949934
3,3,276.340118
4,4,882.033691


In [62]:
sub.dtypes

row_id             int32
meter_reading    float32
dtype: object

In [63]:
sub["meter_reading"].describe()

count    4.169760e+07
mean     2.514620e+02
std      5.448538e+02
min      1.076890e-01
25%      2.191563e+01
50%      8.033254e+01
75%      2.419007e+02
max      3.234327e+03
Name: meter_reading, dtype: float64

In [64]:
sub.isna().sum()

row_id           0
meter_reading    0
dtype: int64

In [0]:
sub.to_csv("submission.csv", index = False)

In [66]:
gc.collect()

456